<a href="https://colab.research.google.com/github/Amirosein/MilkywayFormation/blob/main/MilkyWay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pre

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io, requests, zipfile

!pip install tsfresh
from tsfresh import extract_features

# plt.rcParams.update({'font.size': 20, 'figsize':(8,6)})

SMALL_SIZE = 12
MEDIUM_SIZE = 15
BIGGER_SIZE = 18

plt.rc('font', size=BIGGER_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

plt.rc('axes', linewidth = 5 )
plt.rc('lines', linewidth = 5 )

plt.rc('lines', markersize = 5 )

plt.rc('figure', figsize=(8,4) )         # Image size

# Import data

In [4]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Convert Dat. files to a CSV Function**


In [29]:
import pandas as pd
from pathlib import Path
import csv
from pathlib import Path

f = open("/content/draft.csv", "x")

def data2csv(path, var):
  files = Path(path).rglob("*.dat")

  i = 0

  for file in files:
    datContent = [i.strip().split() for i in open(file).readlines()]
    id = Path(file).stem
    
    # write it as a new CSV file
    with open("/content/draft.csv", "w") as f:
      writer = csv.writer(f)
      writer.writerows(datContent)

    df = pd.read_csv("/content/draft.csv", names=['time','value','e']) 
    df = df[['time','value']]
    df['id'] = id
    df['kind'] = var

    if i==0:
      data = df
    else:
      data = pd.concat([data,df], axis=0)

    i = 1

  return data

# Converting data

# Anomalous-Cepheids

In [ ]:
!pip install pyunpack
!pip install patool
from pyunpack import Archive
Archive('/content/drive/MyDrive/phot.tar.gz').extractall('/content/drive/MyDrive')

In [30]:
LMC_ACEP_I = data2csv('/content/drive/MyDrive/phot/I', 'I')
LMC_ACEP_V = data2csv('/content/drive/MyDrive/phot/V', 'V')

In [44]:
LMC_ACEP = pd.concat([LMC_ACEP_I,LMC_ACEP_V])
LMC_ACEP.head()

,time,value,id,kind
0,2165.84172,17.760,OGLE-LMC-ACEP-002,I
1,2171.86412,17.346,OGLE-LMC-ACEP-002,I
2,2183.82360,17.499,OGLE-LMC-ACEP-002,I
3,2184.82308,17.548,OGLE-LMC-ACEP-002,I
4,2191.78470,17.603,OGLE-LMC-ACEP-002,I


# Feature Extraction Function

In [39]:
def feat(X, cls):
  m = extract_features(X, column_id="id", column_sort="time",
                               column_kind="kind", column_value="value")
  m['class'] = cls
  return m

In [ ]:
acep = feat(LMC_ACEP, 'ACEP')
acep.head()

# Outputs Encoding

> * **One-Hot Encoding**

In [ ]:
from sklearn.preprocessing import OneHotEncoder

a = np.array(data['class'])
OH_Enc = OneHotEncoder( )
OH_Encoded = OH_Enc.fit_transform(a.reshape(-1,1)).toarray()
OH_Outputs = pd.DataFrame(OH_Encoded)
OH_Outputs.columns = ['OH1','OH2','OH3','OH4','OH5']

data = pd.concat([data,OH_Outputs], axis=1)
data.head()

#a = OH_Enc.inverse_transform(np.array([[1, 0, 0, 0, 0],[0, 1, 0, 0, 0],[0, 0, 1, 0, 0]
             #             ,[0, 0, 0, 1, 0],[0, 0, 0, 0, 1]]))

#print('[1, 0, 0, 0, 0] =',a[0],'\n')
#print('[0, 1, 0, 0, 0] =',a[1],'\n')
#print('[0, 0, 1, 0, 0] =',a[2],'\n')
#print('[0, 0, 0, 1, 0] =',a[3],'\n')
#print('[0, 0, 0, 0, 1] =',a[4],'\n')

> * **Ordinal Encoding**

In [ ]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(data['class'])
Ordinal_Y = le.transform(data['class']) 
Y_O = pd.DataFrame(Ordinal_Y)
Y_O.columns = ['Y']

data = pd.concat([data,Y_O], axis=1)
data.head()

#le.inverse_transform([0, 1, 2, 3, 4])

# Shuffle and Imputation

> * Shuffle

In [ ]:
from sklearn.utils import shuffle

data = shuffle(data, random_state=42)
data = data.reset_index()

X = data.loc[:,'Jmag (1.22u)':'160u flux']
Y = data.loc[:,'y1':'y5']
categories = data['class']

data.head()

> * Iterative Imputer

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

imp = IterativeImputer()
imp_data = imp.fit_transform(X)
X = pd.DataFrame(imp_data, columns= X.columns)